# Lab 3: Policy Search

Write agents able to play Nim, with an arbitrary number of rows and an upper bound on the number of objects that can be removed in a turn (a.k.a., subtraction game).

- Task3.1: An agent using fixed rules based on nim-sum (i.e., an expert system)
- Task3.2: An agent using evolved rules
- Task3.3: An agent using minmax
- Task3.4: An agent using reinforcement learning

In [1]:
import logging
from collections import namedtuple
import random
from copy import deepcopy
from itertools import accumulate
from operator import xor

In [2]:
logging.basicConfig(format="%(message)s", level=logging.INFO)

### The Nim and Nimply classes

In [3]:
Nimply = namedtuple("Nimply", "row, num_objects")

In [4]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self) -> int:
        return self._k

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

In [5]:
def possible_moves(nim: Nim):
    return [Nimply(r, o) for r, c in enumerate(nim.rows) for o in range(1, c + 1) if nim.k is None or o <= nim.k]

## An agent using fixed rules

### The random strategy

The hardcoded agent can use a set of fixed strategies: the first one consists in removing a random number of sticks from a random row, if the row has one or more sticks.

In [6]:
def random_strategy(nim: Nim, nimming=True):
    moves_list = possible_moves(nim)

    if len(moves_list) > 0:
        move = random.choice(moves_list)
        if nimming:
            nim.nimming(move)
        return move
    else:
        logging.info("No more moves allowed!")

### The nim sum

The following description was taken from [here](https://dmf.unicatt.it/~paolini/divulgazione/mateappl/nim/nim.html).

In a game of nim that involves nim heaps where you can take as many objects as you want from any one of the heaps during your turn, you need to be able to compute a nim sum, that characterizes the configuration of the game.

Here's how to do it:

- Express the number of objects in each nim heap as a binary number, with the only digits being 0 and 1.
- Fill out the smaller binary numbers with '0's on the left, if necessary, so that all the numbers have the same number of digits.
- Sum the binary numbers, but do not carry.
- Replace each digit in the sum with the remainder that results when the digit is divided by 2.
- This yields the nim sum.
- To win at nim, always make a move, when possible, that leaves a configuration with a nim sum of 0. If you cannot do this, your opponent has the advantage, and you have to depend on his or her committing an error in order to win.
    - Note that if the configuration you are given has a nim sum not equal to 0, there always is a move that creates a new configuration with a nim sum of 0. However, there are usually also moves that will yield configurations that give nim sums not equal to 0, and you need to avoid making these.
    - Also note that if you are given a configuration that has a nim sum of 0, there is no move that will create a configuration that also has a nim sum of 0.

In this implementation, the nim sum is calculated as the bitwise xor of the rows.

In [7]:
def nim_sum(state: Nim) -> int:
    *_, result = accumulate(state.rows, xor)
    return result

### The nim sum strategy

The second strategy consists in calculating the nim sum of the current board:
- if it is not zero, evaluate some random moves until a zero nim sum is found.
- if it is zero, perform a random move.

In [8]:
def nim_sum_strategy(nim: Nim, nimming=True):
    if nim_sum(nim) != 0:
        moves_list = possible_moves(nim)
        for move in moves_list:
            temp_nim = deepcopy(nim)
            temp_nim.nimming(move)
            if nim_sum(temp_nim) == 0:
                if nimming:
                    nim.nimming(move)
                return move
    random_strategy(nim)

### The not so smart strategy

This strategy emulates an human (that can make errors), and consists in using the nim sum strategy with a 70% probability (by default) or the random strategy otherwise.

In [9]:
def not_so_smart_strategy(nim: Nim, smartness=0.7, nimming=True):
    if random.random() < smartness:
        return nim_sum_strategy(nim, nimming=nimming)
    else:
        return random_strategy(nim, nimming=nimming)

## The match

The play_nim function takes four parameters:
- `n` indicates the number of rows on the board.
- `first_strategy` indicates the method used by the first agent to make a move.
- `second_strategy` indicates the method used by the second agent to make a move.
- `who_starts` indicates who makes the first move; it is an even number (tipically 0) to indicate the first agent and an odd number (tipically 1) to indicate the second agent.

In [10]:
def play_nim(n, first_strategy, second_strategy, who_starts):
    nim = Nim(n)
    #logging.info(f"Initial setting: {nim._rows} - Nim sum: {nim_sum(nim)}")
    step = 1 - (who_starts % 2)
    while nim:
        step = 1 - step
        if step == 0: # first agent's turn
            first_strategy(nim)
            #logging.info(f"First agent's turn: {nim._rows} - Nim sum: {nim_sum(nim)}")
        else: # second agent's turn
            second_strategy(nim)
            #logging.info(f"Second agent's turn: {nim._rows} - Nim sum: {nim_sum(nim)}")
    if step == 0:
        logging.info("--- The first agent won! ---\n")
    else:
        logging.info("--- The second agent won! ---\n")


### First version: second agent's random strategy

In [11]:
logging.info("Game 1:\n N = 5\n Start: First Agent\n First agent: nim sum strategy\n Second agent: random strategy")
play_nim(5, nim_sum_strategy, random_strategy, 0)

logging.info("Game 2:\n N = 5\n Start: Second Agent\n First agent: nim sum strategy\n Second agent: random strategy")
play_nim(5, nim_sum_strategy, random_strategy, 1)

Game 1:
 N = 5
 Start: First Agent
 First agent: nim sum strategy
 Second agent: random strategy
--- The first agent won! ---

Game 2:
 N = 5
 Start: Second Agent
 First agent: nim sum strategy
 Second agent: random strategy
--- The first agent won! ---



### Second version: Everyone is using the nim sum strategy

In [12]:
logging.info("Game 3:\n N = 5\n Start: First Agent\n First agent: nim sum strategy \n Second agent: nim sum strategy")
play_nim(5, nim_sum_strategy, nim_sum_strategy, 0)

logging.info("Game 4:\n N = 5\n Start: Second Agent\n First agent: nim sum strategy\n Second agent: nim sum strategy")
play_nim(5, nim_sum_strategy, nim_sum_strategy, 1)

Game 3:
 N = 5
 Start: First Agent
 First agent: nim sum strategy 
 Second agent: nim sum strategy
--- The first agent won! ---

Game 4:
 N = 5
 Start: Second Agent
 First agent: nim sum strategy
 Second agent: nim sum strategy
--- The second agent won! ---



### Third version: The agent is using the not so smart strategy

This strategy emulates an human (that can make errors), and consists in using the nim sum strategy with a 70% probability or the random strategy otherwise.

In [13]:
logging.info("Game 5:\n N = 5\n Start: First Agent\n First agent: nim sum strategy\n Second agent: not so smart strategy")
play_nim(5, nim_sum_strategy, not_so_smart_strategy, 0)

logging.info("Game 6:\n N = 5\n Start: Second Agent\n First agent: nim sum strategy\n Second agent: not so smart strategy")
play_nim(5, nim_sum_strategy, not_so_smart_strategy, 1)

Game 5:
 N = 5
 Start: First Agent
 First agent: nim sum strategy
 Second agent: not so smart strategy
--- The first agent won! ---

Game 6:
 N = 5
 Start: Second Agent
 First agent: nim sum strategy
 Second agent: not so smart strategy
--- The first agent won! ---



## An agent using evolved rules

### The cook status

It is a dictionary containing some current data, like the list of possible moves and the number of active rows; it also contains the possible strategies like:
- random
- nim sum
- not so smart
- shortest row
- longest row
- first row
- last row
- middle row

In [92]:
def items_to_remove(num_law, num):
    assert num > 0
    if num_law == 0: # all the item
        return num
    elif num_law == 1: # one item
        return 1
    elif num_law == 2: # half of the items
        return int((num+1)/2)
    elif num_law == 3: # one/third of the items
        return int((num+2)/3)
    else: # two/third of the items
        return int((num+2)/3) + 1

def shortest_row_strategy(nim: Nim, num_law=0, nimming=True):
    move = min((x for x in enumerate(nim.rows) if x[1] > 0), key=lambda y: y[1])
    move = Nimply(move[0], items_to_remove(num_law, move[1]))
    if nimming:
        nim.nimming(move)
    return move


def longest_row_strategy(nim: Nim, num_law=0, nimming=True):
    move = max((x for x in enumerate(nim.rows)), key=lambda y: y[1])
    move = Nimply(move[0], items_to_remove(num_law, move[1]))
    if nimming:
        nim.nimming(move)
    return move


def first_row_strategy(nim: Nim, num_law=0, nimming=True):
    move = min((x for x in enumerate(nim.rows) if x[1] > 0), key=lambda y: y[0])
    move = Nimply(move[0], items_to_remove(num_law, move[1]))
    if nimming:
        nim.nimming(move)
    return move

def last_row_strategy(nim: Nim, num_law=0, nimming=True):
    move = max((x for x in enumerate(nim.rows) if x[1] > 0), key=lambda y: y[0])
    move = Nimply(move[0], items_to_remove(num_law, move[1]))
    if nimming:
        nim.nimming(move)
    return move


def middle_row_strategy(nim: Nim, num_law=0, nimming=True):
    _list = list(x for x in enumerate(nim.rows) if x[1 > 0])
    move = _list[int(len(_list) / 2)]
    move = Nimply(move[0], items_to_remove(num_law, move[1]))
    if nimming:
        nim.nimming(move)
    return move
    

In [15]:
def cook_status(nim: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = possible_moves(nim)
    cooked["active_rows_number"] = sum(o > 0 for o in nim.rows)

    cooked["nim_sum"] = nim_sum_strategy(nim, nimming=False)
    cooked["random"] = random_strategy(nim, nimming=False)
    cooked["not_so_smart"] = not_so_smart_strategy(nim, nimming=False)
    cooked["shortest_row"] = shortest_row_strategy(nim, nimming=False)
    cooked["longest_row"] = longest_row_strategy(nim, nimming=False)
    cooked["first_row"] = first_row_strategy(nim, nimming=False)
    cooked["last_row"] = last_row_strategy(nim, nimming=False)
    cooked["middle_row"] = middle_row_strategy(nim, nimming=False)

    return cooked

In [95]:
x = Nim(5)
print(x)
x.nimming((0, 1))
x.nimming((4, 9))
print(x)
a = cook_status(x)
middle_row_strategy(x)


<1 3 5 7 9>
<0 3 5 7 0>


(2, 5)